# Particle Swarm

In [120]:
import csv
import numpy as np
from copy import copy
from functools import partial

def newton(pos, vel, acc, n):
    return pos + vel * n + (n * (n + 1) // 2) * acc

def parse_particles(input_path):
    particles = []
    with open(input_path, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            extract = lambda l: np.array(list(map(int, l[3:-1].rstrip('>').split(','))))
            pos, vel, acc = tuple(map(extract, line))
            f = partial(newton, pos, vel, acc)
            particles.append(f)
    return particles

## Part 1

In [122]:
def init_acc(p):
    return p(2) - 2 * p(1) + p(0)

def init_vel(p):
    return p(1) - p(0)

def init_pos(p):
    return p(0)

def assymp_closest(particles):
    min_to_zero = None
    for i, p in enumerate(particles):
        manhattan = sum(np.abs(init_acc(p)))
        if (min_to_zero is None) or manhattan < min_to_zero:
            index = i
            min_to_zero = manhattan
    return index

### Solution

In [123]:
assymp_closest(particles)

258

## Part 2

In [132]:
from itertools import product
from tqdm import tqdm

def update(particles, t):
    return [hash(frozenset(f(t))) for f in particles]

def root_bounds(pos, vel, acc):
    if acc != 0:
        discr = (vel + 0.5 * acc) ** 2 - 2 * acc * pos
        if np.greater_equal(discr, np.zeros_like(discr)):
            x1 = (-0.5 * acc - vel + np.sqrt(discr)) / (2 * acc)
            x2 = (-0.5 * acc - vel - np.sqrt(discr)) / (2 * acc)
            return max(abs(x1), abs(x2))
        else:
            return 0
    elif vel != 0:
        return max(0, -pos / vel)
    else:
        return 0

def time_to_diverge(p):
    pos, vel, acc = init_pos(p), init_vel(p), init_acc(p)
    discr = (acc - 2 * vel) ** 2 - 8 * acc * pos
    if not all(np.greater_equal(discr, np.zeros_like(discr))):
        return 0
    else:
        x = list(map(lambda i: root_bounds(pos[i], vel[i], acc[i]), range(3)))
        return max(x)

def max_time(particles):
    t_max = 0
    for i, j in product(range(len(particles)), repeat=2):
        if i >= j:
            t = time_to_diverge(lambda x: particles[i](x) - particles[j](x))
            if t > t_max: t_max = t
    return int(t_max) + 1
        
def resolve_collisions(particles, t_max):
    for t in range(t_max):
        positions = update(particles, t)
        same = sorted([i for i, pos in enumerate(positions) if positions.count(pos) > 1], reverse=True)
        for i in same:
            particles.pop(i)
        t += 1
    return len(particles)

### Solution

In [134]:
particles = parse_particles('input.txt')
T = max_time(particles)
resolve_collisions(particles, T)

707